In [ ]:
!pip install pymupdf gradio transformers pinecone groq sentence_transformers


In [ ]:
import pymupdf
doc = pymupdf.open('pizza_description.pdf')

doc_text = ""
for page_num in range(doc.page_count):
    page = doc[page_num]
    doc_text += page.get_text()

doc.close()

print(doc_text)

Pizza and its variants are among the most popular foods in the world. Pizza is sold at a
variety of restaurants, including pizzerias (pizza specialty restaurants), Mediterranean
restaurants, via delivery, and as street food. In Italy, pizza served in a restaurant is
presented unsliced, and is eaten with the use of a knife and fork. In casual settings,
however, it is typically cut into slices to be eaten while held in the hand. Pizza is also sold in
grocery stores in a variety of forms, including frozen or as kits for self-assembly. They are
then cooked using a home oven. 
Pizza is sold fresh or frozen, and whole or in portion-size slices. Methods have been
developed to overcome challenges such as preventing the sauce from combining with the
dough, and producing a crust that can be frozen and reheated without becoming rigid. There
are frozen pizzas with raw ingredients and self-rising crusts.
In restaurants, pizza can be baked in an oven with fire bricks above the heat source, an
electr

In [ ]:
chunk_size=50
overlap=10

# chunk_size (int): The number of words in each chunk.
# overlap (int): The number of overlapping words between consecutive chunks.

words = doc_text.split()
chunks = []

begin = 0
while begin < len(words):
    end = begin + chunk_size
    # Extract the chunk and join it back into a string
    chunk = ' '.join(words[begin:end])
    chunks.append(chunk)

    # Move to the next chunk, starting from 'chunk_size - overlap' words ahead
    begin += chunk_size - overlap

# Example usage
print(len(chunks))
print(chunks)

11
['Pizza and its variants are among the most popular foods in the world. Pizza is sold at a variety of restaurants, including pizzerias (pizza specialty restaurants), Mediterranean restaurants, via delivery, and as street food. In Italy, pizza served in a restaurant is presented unsliced, and is eaten with the use', 'restaurant is presented unsliced, and is eaten with the use of a knife and fork. In casual settings, however, it is typically cut into slices to be eaten while held in the hand. Pizza is also sold in grocery stores in a variety of forms, including frozen or as kits', 'in a variety of forms, including frozen or as kits for self-assembly. They are then cooked using a home oven. Pizza is sold fresh or frozen, and whole or in portion-size slices. Methods have been developed to overcome challenges such as preventing the sauce from combining with the dough, and', 'as preventing the sauce from combining with the dough, and producing a crust that can be frozen and reheated witho

In [ ]:
from sentence_transformers import SentenceTransformer

# model = SentenceTransformer('all-MiniLM-L6-v2') #supposed to be fast
# model = SentenceTransformer('deberta-v3-base') #supposed to be pretty good
model = SentenceTransformer('roberta-base')

embeddings = [model.encode(chunk) for chunk in chunks]
print(embeddings)


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


[array([ 2.69476231e-02,  8.34711567e-02,  2.75097080e-02, -9.74950790e-02,
        8.83970782e-02,  6.52812496e-02,  4.38674912e-02,  2.51910090e-02,
       -3.11275665e-02, -8.57947918e-04, -1.85846686e-02, -1.92013502e-01,
        5.54710329e-02,  3.06784827e-02, -3.15234549e-02,  1.55949732e-02,
       -2.69657075e-02,  3.41740511e-02,  4.33719568e-02,  1.36429951e-01,
       -4.12222184e-02,  1.22590452e-01,  7.23133981e-02,  2.06265263e-02,
       -1.93365887e-01,  4.21024337e-02,  1.77530684e-02, -4.48543951e-02,
        9.45463032e-02, -2.35675722e-02, -1.19493909e-01, -3.53884436e-02,
        4.65319604e-02, -2.62047742e-02,  1.00725487e-01,  6.36506975e-02,
        4.14466187e-02,  8.05297717e-02,  3.33980083e-01, -1.36399604e-02,
        4.13581319e-02, -9.25359502e-02, -5.39483614e-02,  2.75238771e-02,
        3.50550935e-02, -8.83849431e-03,  1.43806577e-01,  1.28927723e-01,
        3.25756893e-02, -8.44493601e-03, -5.36969751e-02,  3.53425322e-03,
        9.46908519e-02, 

In [ ]:
import os
import time
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key="pcsk_72fda6_QXrHRdFRxS2SU8oKKL7yMuU4Wg5sE46QEuMJT9e7is4iLueVUMgAyoW16C1YmNL")

index_name = "my-index"


In [ ]:
pc.delete_index(index_name)

In [ ]:
if not pc.has_index(index_name):
    pc.create_index(
        name=index_name,
        # dimension=384,
        dimension=768, #for roberta
        metric="cosine",
        spec=ServerlessSpec(
            cloud='aws',
            region='us-east-1'
        )
    )

while not pc.describe_index(index_name).status['ready']:
    time.sleep(1)

In [ ]:
index = pc.Index(index_name)

records = []
for i, (chunk, embedding) in enumerate(zip(chunks, embeddings)):
    records.append({
        "id": f"chunk_{i}",  # Unique ID for each chunk
        "values": embedding.tolist(),  # Convert embedding to list
        "metadata": {"text": chunk}    # Attach text chunk as metadata
    })

index.upsert(vectors=records, namespace="example-namespace")
print("Embeddings uploaded successfully.")

Embeddings uploaded successfully.


In [ ]:
from groq import Groq
import numpy as np

groq_api_key = "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"
groq_client = Groq(api_key=groq_api_key)

In [ ]:
def get_response_from_groq(context, query):
    messages = [
        {
            "role": "system",
            "content": f"You are a helpful assistant answering questions based on the following context:\n\n{context}"
        },
        {
            "role": "user",
            "content": query
        }
    ]

    response = groq_client.chat.completions.create(
        messages=messages,
        model="llama3-70b-8192"
    )

    return response.choices[0].message.content

In [ ]:
def retrieve_and_answer(query, top_k=3):
    """
    Retrieve relevant chunks and generate an answer using GROQ

    Args:
        query (str): User's question
        top_k (int): Number of relevant chunks to retrieve
    """
    # Create embedding for the query
    query_embedding = model.encode(query)

    # Search in Pinecone
    search_results = index.query(
        vector=query_embedding.tolist(),
        top_k=top_k,
        namespace="example-namespace",
        include_metadata=True
    )

    relevant_chunks = [match.metadata["text"] for match in search_results.matches]
    context = "\n".join(relevant_chunks)

    answer = get_response_from_groq(context, query)

    return {
        "answer": answer,
        "relevant_chunks": relevant_chunks,
        "context": context
    }

In [ ]:
query = "What is the document about?"
result = retrieve_and_answer(query)

print("Answer:", result["answer"])
print("\nRelevant chunks used:")
for i, chunk in enumerate(result["relevant_chunks"], 1):
    print(f"\nChunk {i}:")
    print(chunk)

Answer: The document appears to be about pizza, specifically about making pizza at home, including tips for cooking it in a regular oven to achieve a good crust, and ways to purchase and assemble pizza at home.

Relevant chunks used:

Chunk 1:
sauce and basic ingredients, to assemble at home before baking in an oven.

Chunk 2:
When made at home, a pizza can be baked on a pizza stone in a regular oven to reproduce some of the heating effect of a brick oven. Cooking directly on a metal surface results in too rapid heat transfer to the crust, burning it. Some home chefs use a

Chunk 3:
in a variety of forms, including frozen or as kits for self-assembly. They are then cooked using a home oven. Pizza is sold fresh or frozen, and whole or in portion-size slices. Methods have been developed to overcome challenges such as preventing the sauce from combining with the dough, and


In [ ]:
import gradio as gr

def gradio_interface(query):
    result = retrieve_and_answer(query)
    return result["answer"], "\n\n".join(result["relevant_chunks"])

# Gradio interface
interface = gr.Interface(
    fn=gradio_interface,
    inputs=gr.Textbox(label="Ask a question about the document"),
    outputs=[
        gr.Textbox(label="Answer"),
        gr.Textbox(label="Retrieved Chunks")
    ],
    title="Document Q&A",
    description="Ask questions about the uploaded document and get AI-powered answers."
)
interface.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://6ac45d62cc5aaaca3f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
